<p style="color:#e8574d; font-size:30px; font-weight:bold; font-family:Helvetica"> Content </p>

<ul style="font-size:16px; font-family:Helvetica">
    <li><a href='#copyright' style="color:#696969; text-decoration: none">Copyright and License</a></li>
    <li><a href='#modules' style="color:#696969; text-decoration: none">All Necessary Modules</a></li>
    <li><a href='#hood' style="color:#696969; text-decoration: none">Examine Big Bird under the Hood</a></li>
    <li><a href='#cited' style="color:#696969; text-decoration: none">Reference</a></li>
</ul> 



<p style="color:#e8574d; font-size:20px; font-weight:bold; font-family:Helvetica">Copyright and License<a id='copyright'></a></p>

<p style="color:#696969; font-size:14px">License and copyright notice</p>

In [1]:
!pip show bigbird

Name: bigbird
Version: 0.0.1
Summary: Big Bird: Transformers for Long Sequences
Home-page: http://github.com/google-research/bigbird
Author: Google Inc.
Author-email: no-reply@google.com
License: Apache 2.0
Location: /opt/anaconda3/lib/python3.8/site-packages
Requires: rouge-score, tfds-nightly, numpy, tensorflow, tensor2tensor, absl-py, sentencepiece, natsort, tensorflow-text
Required-by: 


<p style="color:#696969; font-size:14px; margin:0px; padding:0px">State changes:</p>
<p style='font-family:monospace; margin-left:20px'>Name:</p>

<p style="color:#e8574d; font-size:20px; font-weight:bold; font-family:Helvetica">All Necessary Modules<a id='module'></a></p>

In [47]:
# Transformers and BigBird
from transformers import BigBirdTokenizer, BigBirdModel
import transformers

# PyTorch manipulation
from torchvision.transforms import ToTensor
import torch

# TensorFlow manipulation and reading
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info=True, as_supervised=True)


In [39]:
# Read both pubmed and arxiv using TensorFlow
pubmed, pubmed_info = tfds.load("scientific_papers/pubmed",with_info=True, as_supervised=True)
#val_pubmed = tfds.load("tfds://scientific_papers/pubmed", split=tfds.Split.VALIDATION, as_supervised=True)
#test_pubmed = tfds.load("tfds://scientific_papers/pubmed", split=tfds.Split.TEST, as_supervised=True)

INFO:absl:Load dataset info from /Users/jonahwinninghoff/tensorflow_datasets/scientific_papers/pubmed/1.1.1
INFO:absl:Reusing dataset scientific_papers (/Users/jonahwinninghoff/tensorflow_datasets/scientific_papers/pubmed/1.1.1)
INFO:absl:Constructing tf.data.Dataset scientific_papers for split None, from /Users/jonahwinninghoff/tensorflow_datasets/scientific_papers/pubmed/1.1.1


In [51]:
pubmed

{'train': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.string)>,
 'validation': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.string)>,
 'test': <PrefetchDataset shapes: ((), ()), types: (tf.string, tf.string)>}

<p style="color:#e8574d; font-size:20px; font-weight:bold; font-family:Helvetica">Examine Big Bird under the Hood<a id='hood'></a></p>

<p style="color:#e8574d; font-size:20px; font-weight:bold; font-family:Helvetica">Useful Reference<a id='cited'></a></p>

- https://huggingface.co/blog/big-bird
- https://www.tensorflow.org/datasets/catalog/scientific_papers
- http://github.com/google-research/bigbird
- https://proceedings.neurips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf